In [4]:
import os

# Specify the new directory path
new_location = r'C:\Notebooks\mlp_wav2vec'

# Change the current working directory
try:
    os.chdir(new_location)
    print(f"Changed working directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory {new_location} does not exist.")
except Exception as e:
    print(f"Error: {e}")


Changed working directory to: C:\Notebooks\mlp_wav2vec


In [5]:
pwd

'C:\\Notebooks\\mlp_wav2vec'

In [14]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import numpy as np
from tqdm import tqdm
import os

class FrameLevelClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FrameLevelClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

def load_csv_data(file_path):
    df = pd.read_csv(file_path)
    X = df.iloc[:, 1:-2].values  # Drop 'FileID' (first column) and 'Polarity' (last column)
    y = df.iloc[:, -2].values  # 'Label' column (second last)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).unsqueeze(1)

def get_data_loaders(X, y, batch_size, val_split=0.2, test_split=0.1):
    dataset = TensorDataset(X, y)
    val_size = int(len(dataset) * val_split)
    test_size = int(len(dataset) * test_split)
    train_size = len(dataset) - val_size - test_size
    train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)
    test_loader = DataLoader(test_set, batch_size=batch_size)
    return train_loader, val_loader, test_loader

def train_model(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(dataloader, desc="Training", leave=False):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        # outputs = model(X_batch).squeeze()
        # loss = criterion(outputs, y_batch)
        # loss = criterion(outputs, y_batch.squeeze
        outputs = model(X_batch).squeeze().unsqueeze(1)
        loss = criterion(outputs, y_batch)                         
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    all_labels, all_outputs = [], []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch).squeeze().unsqueeze(1)  # Keep as [batch_size, 1]
            # Only use .unsqueeze(1) on y_batch if necessary
            if y_batch.dim() == 1:
                y_batch = y_batch.unsqueeze(1)  # Make y_batch match shape [batch_size, 1]

            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            all_labels.extend(y_batch.squeeze().cpu().numpy())  # Convert back to [batch_size]
            all_outputs.extend(torch.sigmoid(outputs).cpu().numpy().squeeze())  # Convert to [batch_size]
    predictions = (np.array(all_outputs) > 0.5).astype(int)
    accuracy = accuracy_score(all_labels, predictions)
    precision = precision_score(all_labels, predictions)
    recall = recall_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)
    auc = roc_auc_score(all_labels, all_outputs)
    fpr, tpr, thresholds = roc_curve(all_labels, all_outputs)
    fnr = 1 - tpr
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
    cm = confusion_matrix(all_labels, predictions)
    return total_loss / len(dataloader), accuracy, precision, recall, f1, auc, eer, cm





# Main script
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = {
    'hidden_dim': 1024,
    'output_dim': 1,
    'num_epochs': 100,
    'batch_size': 128,
    'learning_rate': 0.001
}

for i in range(1, 7):
    file_path = f"C:\\Notebooks\\rrl_source\\dataset_raw\\train_segment_Wav2Vec2_part{i}.csv"
    X, y = load_csv_data(file_path)
    input_dim = X.shape[1]
    train_loader, val_loader, test_loader = get_data_loaders(X, y, config['batch_size'])
    
    model = FrameLevelClassifier(input_dim, config['hidden_dim'], config['output_dim']).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    best_val_eer = float('inf')
    best_model_path = f"W2V_best_frame_level_model_part{i}.pth"
    result_file = f"performance_part{i}.txt"
    
    for epoch in range(config['num_epochs']):
        train_loss = train_model(model, train_loader, criterion, optimizer)
        val_loss, val_acc, val_prec, val_rec, val_f1, val_auc, val_eer, val_cm = evaluate_model(model, val_loader, criterion)
        
        if val_eer < best_val_eer:
            best_val_eer = val_eer
            torch.save(model.state_dict(), best_model_path)
        
    test_loss, test_acc, test_prec, test_rec, test_f1, test_auc, test_eer, test_cm = evaluate_model(model, test_loader, criterion)
    with open(result_file, 'w') as f:
        f.write(f"Test Loss: {test_loss}\n")
        f.write(f"Accuracy: {test_acc}\n")
        f.write(f"Precision: {test_prec}\n")
        f.write(f"Recall: {test_rec}\n")
        f.write(f"F1 Score: {test_f1}\n")
        f.write(f"AUC: {test_auc}\n")
        f.write(f"EER: {test_eer}\n")
        f.write(f"Confusion Matrix: \n{test_cm}\n")


C:\Users\mawais\miniconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mawais\miniconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mawais\miniconda3\envs\pytorch_env\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\User